In [1]:
# pip install -U datasets pandas numpy scikit-learn gensim nltk shap joblib tqdm matplotlib

In [1]:
import os, re, json, math, warnings, joblib
from dataclasses import dataclass
from typing import List, Tuple, Optional

import numpy as np
import pandas as pd
from tqdm import tqdm

# 데이터
from datasets import load_dataset

# 임베딩/군집
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AffinityPropagation

# 감성
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords

# 모델링/평가
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error

# XAI
import shap
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")


def ensure_nltk():
    """NLTK 리소스 자동 다운로드: VADER, stopwords"""
    try:
        nltk.data.find('sentiment/vader_lexicon.zip')
    except LookupError:
        nltk.download('vader_lexicon')
    try:
        nltk.data.find('corpora/stopwords')
    except LookupError:
        nltk.download('stopwords')


def clean_text(t: str) -> str:
    """공백 정리, 기본 클리닝"""
    return re.sub(r"\s+", " ", str(t)).strip()


def tokenize_en(t: str) -> List[str]:
    """간단 영어 토큰화: 알파벳 토큰 + 불용어 제거"""
    toks = [w.lower() for w in wordpunct_tokenize(t)]
    toks = [w for w in toks if any(c.isalpha() for c in w)]
    sw = set(stopwords.words('english'))
    toks = [w for w in toks if w not in sw and len(w) > 1]
    return toks


@dataclass
class Config:
    w2v_size: int = 100
    ap_affinity: str = "precomputed"  # "precomputed" or "euclidean"
    ap_damping: float = 0.7
    ap_preference: str = "median"     # "median" / "min" / "none"
    random_state: int = 42
    max_train: int = 50000            # 리소스 절약용 샘플 수 (0이면 전체)
    max_test: int = 5000


c:\projects\ai-implementation\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2) 이론 요약 (필요 공식만 간단히)

### 2.1 Affinity Propagation (AP) 핵심 개념
- 유사도 행렬 $S \in \mathbb{R}^{N \times N}$ 사용함 (본 노트북: 코사인 유사도)
- 두 가지 메시지 업데이트함: **책임(responsibility)** $r(i,k)$, **가용성(availability)** $a(i,k)$
- 대표 업데이트 식(직관만 표시):  
$ r(i,k) \leftarrow S(i,k) - \max_{k' \neq k} \{ a(i,k') + S(i,k') \}$  
$ a(i,k) \leftarrow \min\big(0,\; r(k,k) + \sum_{i' \notin \{i,k\}} \max(0, r(i',k)) \big) $
- **preference** 값이 클수록 대표점(exemplar) 수 증가 경향 있음
- **damping** $\in [0.5, 1)$: 수렴 안정성 위해 도입함

### 2.2 VADER 감성
- `compound ∈ [-1, 1]` (전역 극성)
- `pos/neg/neu` 비율도 같이 특성에 포함함

### 2.3 SHAP (KernelExplainer)
- 임의의 블랙박스 함수 $f(\mathbf{x})$에 대해 Shapley 값 근사
- 각 특성 $j$의 중요도 $\phi_j$가 **기여도**로 해석됨
- 회귀에서는 선형 합: $ f(\mathbf{x}) \approx E[f(\mathbf{x})] + \sum_j \phi_j $

### 2.4 Kano 매핑(단순 규칙)
- 클러스터 관련 특성의 **평균 SHAP** 부호/크기/일관성으로 판정함
- 예시 규칙(단순화):
  - $|\overline{\phi}| \approx 0$ → **Indifferent**
  - $\overline{\phi} \gg 0$ → **Attractive / One-dimensional**
  - $\overline{\phi} \ll 0$ → **Must-be / Reverse**
- 실제 연구/프로덕션에서는 임계·신뢰구간·빈도 보정 등 더 정교화 권장함

## 3) 데이터 로딩: MARC (EN)
- Hugging Face `amazon_reviews_multi`, 영어(en) 서브셋 사용함
- 컬럼 예: `review_body`(본문), `review_title`(제목), `stars`(별점 1..5)
- 노트: 이 셀은 인터넷 필요함 (로컬/Colab 등에서 실행 권장)

In [2]:
from datasets import load_dataset

# ✅ 원본(에러 발생): ds = load_dataset("amazon_reviews_multi", "en")
ds = load_dataset("mteb/amazon_reviews_multi", "en")  # ← 미러 사용 (Parquet)

def to_df(split):
    d = ds[split]
    # mteb 스키마: ['id','text','label','label_text']
    # label: 0..4  → 별점(1..5)로 맞춤
    return pd.DataFrame({
        "text": d["text"],
        "stars": (np.array(d["label"]) + 1).tolist()
    })

df_train = to_df("train")
df_test  = to_df("test")


## 4) 전처리 & 토큰화
- 공백 정리, 단어 토큰화, 불용어 제거, 알파벳 토큰만 사용
- Word2Vec 학습 대비

In [3]:

train_texts = [clean_text(t) for t in df_train["text"].tolist()]
test_texts  = [clean_text(t) for t in df_test["text"].tolist()]

tok_train = [tokenize_en(t) for t in tqdm(train_texts, desc="Tokenize train")]
tok_test  = [tokenize_en(t) for t in tqdm(test_texts,  desc="Tokenize test")]

len(tok_train), len(tok_test)


Tokenize test: 100%|██████████| 5000/5000 [00:00<00:00, 9068.73it/s]


(200000, 5000)

## 5) Word2Vec 학습
- 파라미터: `vector_size = cfg.w2v_size`, `window=5`, `min_count=2`
- 코퍼스 전체에 대해 학습 후, 단어 벡터 취득

In [4]:
cfg = Config()
ensure_nltk()
sia = SentimentIntensityAnalyzer()

In [5]:

w2v = Word2Vec(
    sentences=tok_train,
    vector_size=cfg.w2v_size,
    window=5,
    min_count=2,
    workers=4,
    seed=cfg.random_state
)

vocab = list(w2v.wv.index_to_key)
X = np.vstack([w2v.wv[w] for w in vocab])
X.shape, vocab[:10]


((31166, 100),
 ['great',
  'good',
  'one',
  'like',
  'product',
  'would',
  'use',
  'well',
  'quality',
  'get'])

## 6) Affinity Propagation(AP) 군집
- 기본: 코사인 유사도 행렬 $S$ 계산 후 `affinity="precomputed"`로 AP 수행
- `preference`: 기본 median 사용 → 클러스터 수 자동 조절됨

In [ ]:
import torch
from tqdm import tqdm

def build_clusters_with_AP(w2v, affinity="precomputed", damping=0.7, preference="median", random_state=42, use_gpu=True):
    vocab = list(w2v.wv.index_to_key)
    
    # 1단계: 벡터 준비
    with tqdm(total=len(vocab), desc="벡터 준비 중", unit="word") as pbar:
        X = np.vstack([w2v.wv[w] for w in vocab])
        pbar.update(len(vocab))
    
    device = torch.device("cuda" if torch.cuda.is_available() and use_gpu else "cpu")
    
    if affinity == "precomputed":
        # 2단계: GPU로 전송
        with tqdm(total=1, desc="GPU로 데이터 전송 중") as pbar:
            X_torch = torch.from_numpy(X).float().to(device)
            pbar.update(1)
        
        # 3단계: 정규화
        with tqdm(total=1, desc="벡터 정규화 중") as pbar:
            X_norm = torch.nn.functional.normalize(X_torch, p=2, dim=1)
            pbar.update(1)
        
        # 4단계: 코사인 유사도 행렬 계산
        N = X.shape[0]
        with tqdm(total=1, desc=f"코사인 유사도 행렬 계산 중 ({N}x{N})", unit="matrix") as pbar:
            S_gpu = torch.mm(X_norm, X_norm.T)
            pbar.update(1)
        
        # 5단계: CPU로 전송
        with tqdm(total=1, desc="CPU로 결과 전송 중") as pbar:
            S = S_gpu.cpu().numpy()
            pbar.update(1)
        
        # 6단계: Preference 계산
        with tqdm(total=1, desc="Preference 계산 중") as pbar:
            if preference == "median":
                pref = np.median(S)
            elif preference == "min":
                pref = np.min(S)
            else:
                pref = None
            pbar.update(1)
        
        # 7단계: Affinity Propagation 실행
        with tqdm(total=1, desc="Affinity Propagation 실행 중 (sklearn)") as pbar:
            ap = AffinityPropagation(affinity="precomputed", damping=damping, preference=pref, random_state=random_state)
            labels = ap.fit_predict(S)
            pbar.update(1)
    else:
        # Euclidean 모드
        with tqdm(total=1, desc="Affinity Propagation 실행 중 (Euclidean)") as pbar:
            ap = AffinityPropagation(affinity="euclidean", damping=damping, random_state=random_state)
            labels = ap.fit_predict(X)
            pbar.update(1)

    # 8단계: 결과 정리
    with tqdm(total=len(vocab), desc="클러스터 맵 생성 중", unit="word") as pbar:
        cluster_vocab = {w: int(c) for w, c in zip(vocab, labels)}
        pbar.update(len(vocab))
    
    n_clusters = len(set(labels))
    
    return cluster_vocab, n_clusters

cluster_vocab, n_clusters = build_clusters_with_AP(
    w2v, 
    affinity=cfg.ap_affinity, 
    damping=cfg.ap_damping, 
    preference=cfg.ap_preference, 
    random_state=cfg.random_state,
    use_gpu=True  # GPU 사용 여부
)
n_clusters

Affinity Propagation 실행 중 (sklearn):   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:

# def build_clusters_with_AP(w2v, affinity="precomputed", damping=0.7, preference="median", random_state=42):
#     vocab = list(w2v.wv.index_to_key)
#     X = np.vstack([w2v.wv[w] for w in vocab])

#     if affinity == "precomputed":
#         S = cosine_similarity(X)
#         if preference == "median":
#             pref = np.median(S)
#         elif preference == "min":
#             pref = np.min(S)
#         else:
#             pref = None
#         ap = AffinityPropagation(affinity="precomputed", damping=damping, preference=pref, random_state=random_state)
#         labels = ap.fit_predict(S)
#     else:
#         ap = AffinityPropagation(affinity="euclidean", damping=damping, random_state=random_state)
#         labels = ap.fit_predict(X)

#     cluster_vocab = {w: int(c) for w, c in zip(vocab, labels)}
#     n_clusters = len(set(labels))
#     return cluster_vocab, n_clusters

# cluster_vocab, n_clusters = build_clusters_with_AP(
#     w2v, affinity=cfg.ap_affinity, damping=cfg.ap_damping, preference=cfg.ap_preference, random_state=cfg.random_state
# )
# n_clusters


## 7) 특징 벡터 구성
- 클러스터 히스토그램(정규화) + VADER 감성 + 길이/문장부호
- 결과: `[n_samples, n_clusters + 4 + 4]` 크기 특성 행렬

In [ ]:

def featurize_reviews(tokenized_docs, raw_texts, cluster_vocab, n_clusters, sia):
    feats = []
    for toks, raw in zip(tokenized_docs, raw_texts):
        # 1) 클러스터 히스토그램
        hist = np.zeros(n_clusters, dtype=float)
        for t in toks:
            cid = cluster_vocab.get(t, None)
            if cid is not None and cid >= 0:
                hist[cid] += 1.0
        if hist.sum() > 0:
            hist = hist / hist.sum()

        # 2) 감성 (VADER)
        s = sia.polarity_scores(raw)
        sent = np.array([s['pos'], s['neg'], s['neu'], s['compound']], dtype=float)

        # 3) 길이/문장부호
        style = np.array([len(raw), len(toks), raw.count('!'), raw.count('?')], dtype=float)

        feats.append(np.concatenate([hist, sent, style]))
    return np.vstack(feats)

X_train = featurize_reviews(tok_train, train_texts, cluster_vocab, n_clusters, sia)
X_test  = featurize_reviews(tok_test,  test_texts,  cluster_vocab, n_clusters, sia)

feature_names = [f"cluster_{i}" for i in range(n_clusters)] +                 ["sent_pos","sent_neg","sent_neu","sent_compound","len_chars","len_tokens","exclaim_cnt","question_cnt"]

X_train.shape, X_test.shape, feature_names[:8]


## 8) 신경망 학습 (MLP Regressor)
- 타깃: 별점(1..5) → 회귀로 학습, 지표는 **MAE**
- 파이프라인: `StandardScaler(with_mean=False) + MLPRegressor(128,64)`

In [ ]:

y_train = df_train["stars"].astype(float).to_numpy()
y_test  = df_test["stars"].astype(float).to_numpy()

model = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),
    ("mlp", MLPRegressor(hidden_layer_sizes=(128,64), activation="relu",
                         learning_rate_init=1e-3, random_state=cfg.random_state,
                         early_stopping=True, validation_fraction=0.1, max_iter=200))
])

model.fit(X_train, y_train)
pred = model.predict(X_test)
mae = mean_absolute_error(y_test, pred)
print(f"[MAE] test = {mae:.4f}")


## 9) SHAP (KernelExplainer)
- 배경 샘플: 학습 특성에서 일부 샘플링
- 회귀 모델 예측 함수를 그대로 사용
- 요약 플롯 저장함(환경에 따라 노트북 내 표시도 가능)

In [ ]:

bg = shap.sample(X_train, min(100, X_train.shape[0]), random_state=cfg.random_state)

def predict_fn(Xm):
    return model.predict(Xm)

explainer = shap.KernelExplainer(predict_fn, bg)
sample = X_test[: min(400, X_test.shape[0])]
shap_vals = explainer.shap_values(sample, nsamples=200)  # (N_sample, F)

# 요약 플롯
shap.summary_plot(shap_vals, sample, feature_names=feature_names, show=False)
plt.tight_layout()
os.makedirs("artifacts", exist_ok=True)
plt.savefig("artifacts/shap_summary.png", dpi=180)
plt.close()
"artifacts/shap_summary.png 저장됨"


## 10) Kano 규칙 매핑 (간단 버전)
- 평균 SHAP 기반 단순 규칙으로 클러스터별 Kano 후보 판정
- 실제 적용 시 임계/신뢰구간/빈도 보정 등 정교화 권장

In [ ]:

def kano_from_shap(shap_values, feature_names,
                   cluster_prefix="cluster_", neg_thresh=-0.02, pos_thresh=0.02):
    mean_shap = np.mean(shap_values, axis=0)  # (F,)
    kano_map = {}
    for i, fn in enumerate(feature_names):
        if fn.startswith(cluster_prefix):
            v = float(mean_shap[i])
            if abs(v) < 0.005:
                kano = "indifferent"
            elif v > pos_thresh:
                kano = "attractive/one-dimensional"
            elif v < neg_thresh:
                kano = "must-be/reverse"
            else:
                kano = "indifferent"
            kano_map[fn] = {"mean_shap": v, "kano": kano}
    return kano_map

kano_map = kano_from_shap(shap_vals, feature_names)
kano_df = (pd.DataFrame.from_dict(kano_map, orient="index")
           .reset_index().rename(columns={"index":"feature"})
           .sort_values(by="mean_shap", ascending=False))
kano_df.head(10)


## 11) 산출물 저장
- 모델 스택: `artifacts/model_ap.joblib` (파이프라인, Word2Vec, 군집 맵 등)
- SHAP 값/플롯: `artifacts/shap_values.npy`, `artifacts/shap_summary.png`
- Kano 테이블: `artifacts/kano_map.json`

In [ ]:

artifacts = {
    "pipeline": model,
    "w2v": w2v,
    "cluster_vocab": cluster_vocab,
    "n_clusters": n_clusters,
    "feature_names": feature_names
}
os.makedirs("artifacts", exist_ok=True)
joblib.dump(artifacts, "artifacts/model_ap.joblib")
np.save("artifacts/shap_values.npy", shap_vals)
with open("artifacts/kano_map.json","w", encoding="utf-8") as f:
    json.dump({k: {"mean_shap": float(v["mean_shap"]), "kano": v["kano"]} for k,v in kano_map.items()},
              f, ensure_ascii=False, indent=2)
["artifacts/model_ap.joblib", "artifacts/shap_values.npy", "artifacts/kano_map.json"]


## 12) 다음 스텝
- 한국어로 확장하려면: MARC-ko, 한국어 형태소 분석기/감성사전 교체 필요함
- Kano 규칙 고도화: 평균±신뢰구간, 표본빈도 가중, 분포 비교, 다중가설 보정 적용 권장
- AP vs X-means/다른 군집 비교: DBI, NMI, 안정성(다회 실행) 지표로 비교 가능함